In [ ]:
import polars as pl
from zipfile import ZipFile

In [ ]:
with ZipFile(p) as f:
    filename = f.namelist()[0]
    lines = f.open(filename).readlines()
if lines[0].decode().startswith('calc_time'):
    print(lines[0])
    lines = lines[1:]

columns = ['candle_begin_time', 'funding_interval_hours', 'funding_rate']
schema = {
    'candle_begin_time': pl.Int64,
    'funding_rate': pl.Float64,
}
df_lazy = pl.scan_csv(lines, has_header=False, new_columns=columns, schema_overrides=schema)
df_lazy = df_lazy.drop('funding_interval_hours')
candle_begin_time = pl.col('candle_begin_time') - pl.col('candle_begin_time') % (60 * 60 * 1000)
df_lazy = df_lazy.with_columns(candle_begin_time.cast(pl.Datetime('ms')).dt.replace_time_zone('UTC'))

df_lazy.collect()